In [ ]:
import os
import sys
import pandas as pd
import seaborn as sns
sys.path.append(os.path.abspath(os.path.join('..', 'src', 'utils')))
import utils as ut

In [6]:
wines_df = pd.read_csv("../src/data/processed/all_wines.csv")
wines_df

,wine_link,name,year,winery,rating,rating_qty,price,body,tannis,sweetness,acidity,notes,pairings,grapes,region,style,image
0,https://www.vivino.com/US/en/luigi-bosca-parai...,Paraiso,2020,Luigi Bosca,4.8,575,164.99,73.43435%,50.905894999999994%,13.619732499999998%,44.74824%,"{'black fruit': 14, 'oaky': 13, 'earthy': 5, '...","['beef', 'lamb', 'poultry', 'game (deer, venis...","['Cabernet Sauvignon', 'Malbec']","['Argentina', 'Mendoza']",Argentinian Cabernet Sauvignon - Malbec,https://images.vivino.com/thumbs/_Bf6JTwYRpSX6...
1,https://www.vivino.com/US/en/catena-zapata-est...,Estiba Reservada,2015,Catena Zapata,4.7,295,675.00,74.09986749999999%,55.86741500000001%,14.329525%,54.65305%,"{'oaky': 109, 'black fruit': 50, 'spices': 25,...","['beef', 'lamb', 'poultry', 'pasta']","['Cabernet Sauvignon', 'Malbec', 'Merlot']","['Argentina', 'Mendoza', 'Agrelo']",Argentinian Bordeaux Blend,https://images.vivino.com/thumbs/Yt464jw0QS-ug...
2,https://www.vivino.com/US/en/catena-zapata-est...,Estiba Reservada,2017,Catena Zapata,4.7,218,580.00,74.09986749999999%,55.86741500000001%,14.329525%,54.65305%,"{'oaky': 109, 'black fruit': 50, 'spices': 25,...","['beef', 'lamb', 'poultry', 'pasta']","['Cabernet Sauvignon', 'Malbec', 'Merlot']","['Argentina', 'Mendoza', 'Agrelo']",Argentinian Bordeaux Blend,https://images.vivino.com/thumbs/Yt464jw0QS-ug...
3,https://www.vivino.com/US/en/monteviejo-la-vio...,La Violeta,2013,Monteviejo,4.7,202,150.00,73.06289000000001%,40.9165975%,21.4814425%,36.61685%,"{'oaky': 51, 'black fruit': 49, 'earthy': 26, ...","['beef', 'lamb']",['100%Malbec'],"['Argentina', 'Mendoza', 'Uco Valley']",Argentinian Uco Valley Malbec Red,https://images.vivino.com/thumbs/FO-x9h3mQHSx9...
4,https://www.vivino.com/US/en/vina-cobos-cobos-...,Cobos Volturno,2018,Viña Cobos,4.7,198,399.99,71.05489250000001%,45.630624999999995%,24.2544575%,38.45142%,"{'oaky': 86, 'black fruit': 49, 'spices': 17, ...","['beef', 'lamb', 'poultry', 'game (deer, venis...",['100%Cabernet Sauvignon'],"['Argentina', 'Mendoza', 'Perdriel']",Argentinian Cabernet Sauvignon - Malbec,https://images.vivino.com/thumbs/D1Mf1fYnRnage...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,https://www.vivino.com/etchart-cafayate-terroi...,Cafayate Terroir de Altura Torrontés,2021,Bodegas Etchart,3.9,110,NaN,47.892224999999996%,NaN,19.515900000000002%,56.4788025%,"{'tree fruit': 3, 'red fruit': 1, 'floral': 1}","['vegetarian', 'spicy food', 'aperitif']",['Torrontés'],"['Argentina', 'Salta', 'Calchaqui Valley', 'Ca...",Argentinian Torrontés,https://images.vivino.com/thumbs/zbf30yF2Q_6j_...
2022,https://www.vivino.com/siesta-cabernet-sauvign...,Cabernet Sauvignon,2011,Siesta,3.9,108,NaN,65.158975%,54.17996%,13.757972500000001%,41.570055%,"{'oaky': 22, 'black fruit': 14, 'red fruit': 1...","['beef', 'lamb', 'poultry']",['100%Cabernet Sauvignon'],"['Argentina', 'Mendoza']",Argentinian Cabernet Sauvignon,https://images.vivino.com/thumbs/hz2-G9-7TyGMC...
2023,https://www.vivino.com/gen-del-alma-jijiji-mal...,JIJIJI Malbec - Pinot Noir,2022,Gen del Alma,3.9,105,22.66,63.11037999999999%,45.074575%,21.187649999999998%,58.78908%,"{'red fruit': 60, 'black fruit': 27, 'earthy':...","['beef', 'lamb', 'pork', 'poultry', 'mushrooms...","['Malbec', 'Pinot Noir']","['Argentina', 'Mendoza', 'Uco Valley', 'Tupung...",Argentinian Malbec Red Blend,https://images.vivino.com/thumbs/hqHY7FTaRoucR...
2024,https://www.vivino.com/terrazas-de-los-andes-h...,High Altitude Vineyards Malbec,2021,Terrazas de los Andes,3.9,103,NaN,62.6411375%,31.788999999999994%,18.47617%,37.5830825%,"{'oaky': 1756, 'black fruit': 1668, 'earthy': ...","['beef', 'lamb']",['100%Malbec'],"['Argentina', 'Mendoza', 'Lujan de Cuyo']",Argentinian Lujan de Cuyo Malbec Red,https://images.vivino.com/thumbs/xaqIkl7ERfa2e...


In [8]:
wines_df.shape

(2026, 17)

In [9]:
wines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2026 entries, 0 to 2025
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   wine_link   2026 non-null   object 
 1   name        2023 non-null   object 
 2   year        2023 non-null   object 
 3   winery      2026 non-null   object 
 4   rating      2026 non-null   float64
 5   rating_qty  2026 non-null   object 
 6   price       1601 non-null   float64
 7   body        1974 non-null   object 
 8   tannis      1663 non-null   object 
 9   sweetness   1945 non-null   object 
 10  acidity     1974 non-null   object 
 11  notes       2026 non-null   object 
 12  pairings    2026 non-null   object 
 13  grapes      2026 non-null   object 
 14  region      2026 non-null   object 
 15  style       2021 non-null   object 
 16  image       2026 non-null   object 
dtypes: float64(2), object(15)
memory usage: 269.2+ KB


In [ ]:
null_tannis = wines_df[wines_df["tannis"].isna()]
ut.save_csv(null_tannis, "/")

,wine_link,name,year,winery,rating,rating_qty,price,body,tannis,sweetness,acidity,notes,pairings,grapes,region,style,image
81,https://www.vivino.com/catena-zapata-adrianna-...,Adrianna Vineyard White Bones Chardonnay,2014,Catena Zapata,4.4,337,160.01,56.52603500000001%,NaN,15.200217499999997%,45.27974%,"{'citrus': 251, 'earthy': 244, 'tree fruit': 2...","['pork', 'vegetarian', 'poultry', 'rich fish (...",['100%Chardonnay'],"['Argentina', 'Mendoza']",Argentinian Chardonnay,https://images.vivino.com/thumbs/RrYTlfu7QdmGj...
84,https://www.vivino.com/catena-zapata-adrianna-...,Adrianna Vineyard White Bones Chardonnay,2017,Catena Zapata,4.4,308,118.95,56.52603500000001%,NaN,15.200217499999997%,45.27974%,"{'citrus': 251, 'earthy': 244, 'tree fruit': 2...","['pork', 'vegetarian', 'poultry', 'rich fish (...",['100%Chardonnay'],"['Argentina', 'Mendoza']",Argentinian Chardonnay,https://images.vivino.com/thumbs/RrYTlfu7QdmGj...
85,https://www.vivino.com/catena-zapata-adrianna-...,Adrianna Vineyard White Bones Chardonnay,2018,Catena Zapata,4.4,287,148.95,56.52603500000001%,NaN,15.200217499999997%,45.27974%,"{'citrus': 251, 'earthy': 244, 'tree fruit': 2...","['pork', 'vegetarian', 'poultry', 'rich fish (...",['100%Chardonnay'],"['Argentina', 'Mendoza']",Argentinian Chardonnay,https://images.vivino.com/thumbs/RrYTlfu7QdmGj...
87,https://www.vivino.com/catena-zapata-adrianna-...,Adrianna Vineyard White Stones Chardonnay,2020,Catena Zapata,4.4,270,139.98,51.047265%,NaN,13.4267225%,47.167365000000004%,"{'tree fruit': 245, 'citrus': 238, 'earthy': 2...","['pork', 'vegetarian', 'poultry', 'rich fish (...",['100%Chardonnay'],"['Argentina', 'Mendoza']",Argentinian Chardonnay,https://images.vivino.com/thumbs/moA8OTV-QeqK1...
88,https://www.vivino.com/catena-zapata-adrianna-...,Adrianna Vineyard White Stones Chardonnay,2021,Catena Zapata,4.4,220,146.99,51.047265%,NaN,13.4267225%,47.167365000000004%,"{'tree fruit': 245, 'citrus': 238, 'earthy': 2...","['pork', 'vegetarian', 'poultry', 'rich fish (...",['100%Chardonnay'],"['Argentina', 'Mendoza']",Argentinian Chardonnay,https://images.vivino.com/thumbs/moA8OTV-QeqK1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1993,https://www.vivino.com/mendel-winery-semillon/...,Sémillon,2020,Mendel,3.9,166,33.33,40.8859175%,NaN,10.145657499999999%,51.21259%,"{'citrus': 208, 'earthy': 186, 'tree fruit': 1...","['pork', 'shellfish', 'rich fish (salmon, tuna...",['100%Sémillon'],"['Argentina', 'Mendoza']",Argentinian Sémillon White,https://images.vivino.com/thumbs/MjV2YvZiTmyw6...
2010,https://www.vivino.com/pulenta-estate-sauvigno...,Sauvignon Blanc (VI),2018,Pulenta Estate,3.9,134,35.00,42.805299999999995%,NaN,0,67.30869999999999%,"{'citrus': 67, 'tree fruit': 46, 'tropical': 2...","['shellfish', 'vegetarian', ""goat's milk cheese""]",['100%Sauvignon Blanc'],"['Argentina', 'Mendoza', 'Agrelo']",Argentinian Sauvignon Blanc,https://images.vivino.com/thumbs/RHGN8lUMRk686...
2011,https://www.vivino.com/phebus-torrontes/w/1153...,Torrontes,2022,Phebus,3.9,133,NaN,54.443055%,NaN,3.0036799999999992%,59.796385%,"{'citrus': 350, 'tree fruit': 182, 'earthy': 8...","['vegetarian', 'spicy food', 'aperitif']",['Torrontés'],"['Argentina', 'Mendoza']",Argentinian Torrontés,https://images.vivino.com/thumbs/K5keQubNRHmGp...
2019,https://www.vivino.com/zorzal-eggo-blanc-de-ca...,Eggo Blanc de Cal,2019,Zorzal,3.9,115,33.76,43.3834%,NaN,0.9501024999999998%,66.745675%,"{'citrus': 53, 'earthy': 38, 'vegetal': 32, 't...","['shellfish', 'vegetarian', ""goat's milk cheese""]",['100%Sauvignon Blanc'],"['Argentina', 'Mendoza', 'Uco Valley', 'Tupung...",Argentinian Sauvignon Blanc,https://images.vivino.com/thumbs/sqrrSTH8QPymG...
